In [1]:
## 1. import libraries
import pandas as pd
import numpy as np

## 2. file path and load csv
FOLDER_PATH = "" ## 파일 경로
file_path = FOLDER_PATH+"lending_club_2020_train.csv"
origin_data = pd.read_csv(file_path)
origin_data.head()

## 3. drop할 컬럼들을 일단 처리한다.
with open('drop_columns.txt', mode='r') as f:
    drop_fields = f.readlines()
    drop_fields = [drop_field.strip('\n') for drop_field in drop_fields]
    
drop_df = origin_data.drop(columns=drop_fields)

# 컬럼 개수 출력
print(len(drop_df.columns)) # 96

## 4. 반응변수 처리
# loan_status가 "current", "issued", "policy" 인 행을 필터링하여 삭제
modified_df = drop_df[~drop_df['loan_status'].isin(['Current', 'Issued', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]

# risk = 1, safe = 0 으로 처리
modified_df.loc[modified_df['loan_status'].isin(['Fully Paid', 'In Grace Period']), 'loan_status'] = 0
modified_df.loc[modified_df['loan_status'].isin(['Charged Off', 'Default', 'Late (16-30 days)', 'Late (31-120 days)']), 'loan_status'] = 1

print(len(modified_df)) # 1133666


## 5. 데이터 처리용 함수
def delete_suffix(term:str)->int:
    '''첫 단어만을 저장하는 함수'''
    term = term.strip().split()[0]
    return int(term)

def delete_suffix_percentage(term:str)->float:
    '''%를 자르는 함수'''
    term = term.strip('%')
    return float(term)

def delete_suffix_xx(term:str)->int:
    '''xx를 자르는 함수'''
    term = term.strip('xx')
    return int(term)

def apply_to_multiple_columns(date_columns:list)->None:
    '''modified_df에서 %b-%Y(Aug-2006) 형식의 날짜 정보를 %Y-%m-%d(2006-08-01)형태로 저장하는 함수'''
    for col in date_columns:
        modified_df[col]=pd.to_datetime(modified_df[col], format='%b-%Y')

C:\Users\Na\AppData\Local\Temp\ipykernel_27460\3147556744.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  origin_data = pd.read_csv(file_path)


79
1133666


In [2]:
THRESHOLD = 100

null_percentages = (modified_df.isnull().sum() * 100) / len(modified_df)
columns_to_drop = null_percentages[null_percentages >= THRESHOLD].index
columns_to_keep = null_percentages[null_percentages < THRESHOLD].index

# 필드 추출
filtered_df = modified_df[columns_to_keep]
filtered_df.columns
print(columns_to_drop)

Index([], dtype='object')


In [3]:
filtered_df = filtered_df.dropna(subset=filtered_df.columns.difference(['loan_status']),how='all')

In [4]:
## 6. 결측 처리
# 6-1) 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
filtered_df = filtered_df.dropna(subset=['chargeoff_within_12_mths','collections_12_mths_ex_med','dti',
                                         'pub_rec_bankruptcies','revol_util','tax_liens'])
print(filtered_df.shape) # (1131682, 79)

# 6-2) 대체하기 어려운 변수 또는 결측이 많은 변수 삭제
filtered_df = filtered_df.drop(columns=['bc_util', 'mths_since_last_delinq','mths_since_last_major_derog',
                                        'mths_since_last_record','mths_since_recent_bc_dlq',
                                        'mths_since_recent_revol_delinq','percent_bc_gt_75'])
print(filtered_df.shape) #(1131682, 72)

# 6-3) 결측 대체
# A. joint 관련 변수 : 결측 0 대체
filtered_df[['annual_inc_joint','dti_joint','revol_bal_joint']] = filtered_df[['annual_inc_joint','dti_joint','revol_bal_joint']].fillna(0)

# B. 최빈값 대체
filtered_df['mo_sin_old_il_acct'] = filtered_df['mo_sin_old_il_acct'].fillna(129)
filtered_df['mths_since_recent_bc'] = filtered_df['mths_since_recent_bc'].fillna(13)
filtered_df['mths_since_recent_inq'] = filtered_df['mths_since_recent_inq'].fillna(5)
filtered_df['bc_open_to_buy'] = filtered_df['bc_open_to_buy'].fillna(5200.0)
filtered_df['emp_length'] = filtered_df['emp_length'].fillna('10+ years')

# C. 2012년 대체
### 업데이트예정 ###


# D. 2015년 대체
### 업데이트 예정 ###



(1131682, 79)
(1131682, 72)


In [ ]:
### 수정중입니다 ###

# 수치형 처리
# 변수 20번
filtered_df['emp_length'] = filtered_df['emp_length'].apply(lambda x: x.replace(' years','').replace(' year','').replace('+','').replace('< ', ''))
# 변수 92번
filtered_df['revol_util'].apply(delete_suffix_percentage)